In [1]:
import torch
from torch_geometric.datasets import MovieLens1M, MovieLens100K


# Load MovieLens 1M dataset (adjust root if needed)
#dataset = MovieLens1M(root='/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg')
dataset= MovieLens100K(root="/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/temp_pyg")
print(len(dataset))


/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


1


In [2]:
import os

In [2]:
graph = dataset[0]
dataset[0].num_nodes
print (dataset.data)


HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)


/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/torch_geometric/data/in_memory_dataset.py:284: UserWarning: It is not recommended to directly access the internal storage format `data` of an 'InMemoryDataset'. If you are absolutely certain what you are doing, access the internal storage via `InMemoryDataset._data` instead to suppress this warning. Alternatively, you can access stacked individual attributes of every graph via `dataset.{attr_name}`.
  warnings.warn(msg)


In [13]:
print(graph)

HeteroData(
  movie={ x=[1682, 18] },
  user={ x=[943, 24] },
  (user, rates, movie)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
    edge_label_index=[2, 20000],
    edge_label=[20000],
  },
  (movie, rated_by, user)={
    edge_index=[2, 80000],
    rating=[80000],
    time=[80000],
  }
)


In [6]:
edge_index = graph['user', 'rates', 'movie'].edge_index
# Assuming 'edge_index' contains [user_ids, movie_ids] and num_nodes is the total number of nodes
num_nodes = dataset[0].num_nodes
user_ids = edge_index[0]
user_max_id = user_ids.max().item()  # Maximum user ID
movie_ids = edge_index[1] +  user_max_id + 1

# Get the maximum number of users (should be user_ids.max()) and total number of nodes

movie_min_id = movie_ids.min().item()  # Minimum movie ID
movie_max_id = movie_ids.max().item()  # Maximum movie ID
print (movie_min_id)
print (movie_max_id)
print (num_nodes)
# Check if the IDs are in the correct range
valid_user_ids = (user_ids >= 0).all() and (user_ids <= user_max_id).all()
valid_movie_ids = (movie_ids > user_max_id).all() and (movie_ids < num_nodes).all()

if valid_user_ids and valid_movie_ids:
    print("All user and movie IDs are within the valid range.")
else:
    if not valid_user_ids:
        print(f"Error: User IDs are out of range. User IDs should be between 0 and {user_max_id}.")
    if not valid_movie_ids:
        print(f"Error: Movie IDs are out of range. Movie IDs should be between {user_max_id + 1} and {num_nodes - 1}.")


943
2624
2625
All user and movie IDs are within the valid range.


In [ ]:
import torch

# Assuming graph is your loaded dataset object
edge_index = graph['user', 'rates', 'movie'].edge_index

# Extract the user IDs and movie IDs from the edge index
# edge_index[0] contains user IDs (source nodes)
# edge_index[1] contains movie IDs (target nodes)
user_ids = edge_index[0].unique()
movie_ids = edge_index[1].unique()

intersection_mask = torch.isin(user_ids, movie_ids)

# Get the actual intersecting IDs
intersection = user_ids[intersection_mask]

if intersection.numel() > 0:
    print(f"Users and movies share the same IDs: {intersection}")
else:
    print("No users share the same IDs with movies.")


In [4]:
# Get the ratings
ratings = dataset[0]['user', 'rates', 'movie'].rating

# Find the unique ratings
unique_ratings = torch.unique(ratings)

# Count the number of unique ratings
num_unique_ratings = unique_ratings.size(0)

# Print the result
print(f'Number of unique ratings: {num_unique_ratings}')
print(f'Unique ratings: {unique_ratings}')


Number of unique ratings: 5
Unique ratings: tensor([1, 2, 3, 4, 5])


In [20]:
#reading the edges
edges = graph['user', 'rates', 'movie'].edge_index

triplets = []
for i in range(edges.size(1)):
    u, v = edges[:, i]  # Get the user and movie node index
    triplets.append((u.item(),0, v.item()))

print(triplets[:10])


[(0, 0, 1176), (0, 0, 655), (0, 0, 902), (0, 0, 3339), (0, 0, 2286), (0, 0, 1179), (0, 0, 1267), (0, 0, 2735), (0, 0, 590), (0, 0, 907)]


In [22]:
import random

random.shuffle(triplets)
num_train = int(0.8 * len(triplets))
num_val = int(0.1 * len(triplets))

train_triplets = triplets[:num_train]
val_triplets = triplets[num_train:num_train + num_val]
test_triplets = triplets[num_train + num_val:]


# Save them into separate files
def save_triplets(filename, triplets):
    with open(filename, "w") as f:
        for u, v, e in triplets:
            f.write(f"{u}\t{v}\t{e}\n")

save_triplets("train.txt", train_triplets)
save_triplets("valid.txt", val_triplets)
save_triplets("test.txt", test_triplets)
print ("done")


done


In [3]:
import os
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 

In [2]:
import os
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
%run script/run.py -c config/transductive/inference.yaml --dataset WN18RR --epochs 0 --bpe null --gpus [0] --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

15:47:42   Random seed: 1024
15:47:42   Config file: config/transductive/inference.yaml
15:47:42   {'checkpoint': '/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth',
 'dataset': {'class': 'WN18RR', 'root': '~/git/ULTRA/kg-datasets/'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func': 'sum',
                              'class': 'RelNBFNet',
                              'hidden_dims': [64, 64, 64, 64, 64, 64],
                              'input_dim': 64,
                              'layer_norm': True,
                              'message_func': 'distmult',
        

edge_index[1]: tensor([10211, 25525,  3891,  ..., 39607, 39608, 39609])
edge_type: tensor([ 3,  9, 10,  ..., 14, 14, 14])
edge_index[1] size: torch.Size([173670])
edge_type size: torch.Size([173670])


/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ultra/tasks.py:200: UserWarning: Sparse CSR tensor support is in beta state. If you miss a functionality in the sparse tensor support, please submit a feature request to https://github.com/pytorch/pytorch/issues. (Triggered internally at /opt/conda/conda-bld/pytorch_1695392022560/work/aten/src/ATen/SparseCsrTensorImpl.cpp:53.)
  Ahh = torch.sparse.mm(EhT, Eh).coalesce()


edge_index[1]: tensor([10211, 25525,  3891,  ..., 39607, 39608, 39609])
edge_type: tensor([ 3,  9, 10,  ..., 14, 14, 14])
edge_index[1] size: torch.Size([173670])
edge_type size: torch.Size([173670])
edge_index[1]: tensor([10211, 25525,  3891,  ..., 39607, 39608, 39609])
edge_type: tensor([ 3,  9, 10,  ..., 14, 14, 14])
edge_index[1] size: torch.Size([173670])
edge_type size: torch.Size([173670])


ValueError: abort.

In [ ]:

%run script/run.py -c config/transductive/inference.yaml --dataset MovieLens100k --epochs 0 --bpe null --gpus [0] --ckpt /usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth

/itet-stor/trachsele/net_scratch/conda_envs/ba_bugfix/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
15:50:28   Random seed: 1024
15:50:28   Config file: config/transductive/inference.yaml
15:50:28   {'checkpoint': '/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec/ckpts/ultra_4g.pth',
 'dataset': {'class': 'MovieLens100k', 'root': '~/git/ULTRA/kg-datasets/'},
 'model': {'class': 'Ultra',
           'entity_model': {'aggregate_func': 'sum',
                            'class': 'EntityNBFNet',
                            'hidden_dims': [64, 64, 64, 64, 64, 64],
                            'input_dim': 64,
                            'layer_norm': True,
                            'message_func': 'distmult',
                            'short_cut': True},
           'relation_model': {'aggregate_func':

rel graph is built


15:50:30   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:50:30   Evaluate on valid


Load rspmm extension. This may take a while...


15:51:23   mr: 204.833
15:51:23   mrr: 0.0520594
15:51:23   hits@1: 0.0203125
15:51:23   hits@3: 0.0455
15:51:23   hits@10: 0.107125
15:51:23   >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
15:51:23   Evaluate on test


In [1]:
import numpy as np
import torch

print(np.__version__)  # Check NumPy version
print(torch.__version__)  # Check PyTorch version
print(torch.cuda.is_available())  # Check if CUDA is available


1.24.3
2.1.0
True


In [2]:
print(os.getcwd())
os.chdir("/usr/itetnas04/data-scratch-01/trachsele/data/tl4rec") 
print(os.getcwd())

NameError: name 'os' is not defined

In [13]:
print(os.listdir())

['valid.txt', 'requirements.txt', 'MovieLense_load.ipynb', 'temp_pyg', 'config', '.git', '.gitignore', 'ultra', 'LICENSE', 'script', 'test.txt', 'train.txt', 'ckpts', 'asset', 'README.md', '.ipynb_checkpoints']


In [4]:
import requests

# URLs of the triplet files
urls = {
    "train": "https://raw.githubusercontent.com/Elematre/tl4rec/refs/heads/main/train.txt",
    "valid": "https://raw.githubusercontent.com/Elematre/tl4rec/refs/heads/main/valid.txt",
    "test": "https://raw.githubusercontent.com/Elematre/tl4rec/refs/heads/main/test.txt",
}

# Function to load triplet data from a URL
def load_triplets(url):
    response = requests.get(url)
    response.raise_for_status()  # Ensure we got the data correctly
    triplets = [line.strip().split() for line in response.text.splitlines()]
    return triplets

# Function to extract the set of unique nodes from triplets
def get_unique_nodes(triplets):
    nodes = set()
    for triplet in triplets:
        u, e, v = triplet
        nodes.add(u)
        nodes.add(v)
    return nodes

# Load triplet data from URLs
train_triplets = load_triplets(urls["train"])
valid_triplets = load_triplets(urls["valid"])
test_triplets = load_triplets(urls["test"])

# Extract unique nodes
train_nodes = get_unique_nodes(train_triplets)
valid_nodes = get_unique_nodes(valid_triplets)
test_nodes = get_unique_nodes(test_triplets)

# Find missing nodes in valid and test datasets
missing_in_valid = valid_nodes - train_nodes
missing_in_test = test_nodes - train_nodes

# Print results
if missing_in_valid:
    print(f"Nodes in valid but not in train: {missing_in_valid}")
else:
    print("All nodes in valid are in train.")

if missing_in_test:
    print(f"Nodes in test but not in train: {missing_in_test}")
else:
    print("All nodes in test are in train.")


All nodes in valid are in train.
All nodes in test are in train.
